In [70]:
import pandas as pd

data = pd.read_csv('data/train.csv')

data.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


In [71]:
single_row = data.iloc[28:29]  # Возвращает DataFrame с одной строкой

In [72]:
single_row

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
28,29,2011-09-26,73,38.0,15.0,NaN,NaN,NaN,NaN,NaN,...,20,3,0,17,35,0,9,98,7,8254400


In [73]:
single_row['product_type']

28    OwnerOccupier
Name: product_type, dtype: object

In [1]:
import joblib
model_Occupier = joblib.load('model_Occupier.pkl')
model_Investment = joblib.load('model_Investment.pkl')

In [17]:
data.columns

Index(['full_sq', 'life_sq', 'floor', 'max_floor', 'material', 'build_year',
       'num_room', 'kitch_sq', 'state', 'area_m',
       ...
       'month_2', 'month_3', 'month_4', 'month_5', 'month_7', 'month_8',
       'month_9', 'month_10', 'month_11', 'month_12'],
      dtype='object', length=177)

In [15]:
single_row.shape

(1, 284)

In [18]:
data.shape

(7662, 177)

In [74]:
import json

def preprocess_data_Occupier(input_data: pd.DataFrame) -> pd.DataFrame:
        
        with open('removed_columns.txt', 'r') as f:
            removed_columns = [line.strip() for line in f.readlines()]

        with open('mean_values.json', 'r') as f:
            mean_values = json.load(f)

        data = pd.DataFrame(input_data)

        data = data.drop([
            'ID_metro',
            'ID_railroad_station_walk',
            'ID_railroad_station_avto',
            'ID_big_road1',
            'ID_big_road2',
            'ID_railroad_terminal',
            'ID_bus_terminal'
        ], axis=1)

        data = data.drop(columns=removed_columns, errors='ignore')

        data = data.drop('id', axis=1)

        numeric_columns = data.loc[:, data.dtypes!='object'].columns

        for col in numeric_columns:
            if col in mean_values:
                data[col] = data[col].fillna(mean_values[col])

        categorical_columns = data.loc[:, data.dtypes == 'object'].columns

        label_encoder = LabelEncoder()

        # Применяем Label Encoding для категориальных столбцов
        for col in categorical_columns:
            if col != 'timestamp':
                # Применяем LabelEncoder
                data[col] = label_encoder.fit_transform(data[col])
        
        data['timestamp'] = pd.to_datetime(data['timestamp'])

        data['month'] = data.timestamp.dt.month
        data['year'] = data.timestamp.dt.year

        data = data.sort_values(['timestamp'])
        
        data['year'] = label_encoder.fit_transform(data['year'])

        data['year'] = label_encoder.fit_transform(data['month'])

        data = data.drop('timestamp', axis=1)

        Occupier = data[data['product_type'] == 0].copy()

        return Occupier


In [75]:
single_row = preprocess_data_Occupier(single_row)

C:\Users\denis\AppData\Local\Temp\ipykernel_21168\2676033080.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['month'] = data.timestamp.dt.month
C:\Users\denis\AppData\Local\Temp\ipykernel_21168\2676033080.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['year'] = data.timestamp.dt.year


In [76]:
single_row.shape

(1, 151)

In [77]:
single_row

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,...,green_part_5000,prom_part_5000,office_count_5000,trc_sqm_5000,cafe_sum_5000_min_price_avg,mosque_count_5000,market_count_5000,price_doc,month,year
28,73,38.0,15.0,12.558974,1.827121,3068.057097,1.909804,6.399301,2.107025,0,...,41.97,11.03,76,830817,706.57,0,7,8254400,9,0


In [78]:
import numpy as np

single_row = single_row.drop('price_doc', axis=1)

def load_model(path: str):
        import joblib
        return joblib.load(path)

test_model = load_model('xg_model_Occupier.pkl')

print("Предсказание: ", np.exp(test_model.predict(single_row.head(1))))

Предсказание:  [8526827.]
